In [42]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter 
from wordcloud import WordCloud, STOPWORDS 
import re 
import string
import nltk # preprocessing text
from textblob import TextBlob
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import statsmodels.api as sm



In [2]:
df=pd.read_csv('reviews_table.csv')
reviewsdf=df.copy()

In [3]:
bdf=pd.read_csv('business_table.csv')
businessdf=bdf.copy()

BUSINESS FILE CLEANUP

In [12]:
# cleaning business table

#creating a copy and dropping nulls
newdf=businessdf.copy()
newdf=newdf.dropna()


#Split attributes into different columns
def create_attributes(df, dictList):
    
    for dictionaryColumn in dictList:
        
        #the attributes column is a string of dictionaries, so one extra step is taken to convert
        if dictionaryColumn == 'attributes':
            expandedColumns = df[dictionaryColumn].map(eval).apply(pd.Series)
        else:
            expandedColumns = df[dictionaryColumn].map(eval).apply(pd.Series)
        
        df = pd.concat([df.drop(dictionaryColumn,axis=1), 
                   expandedColumns]
                  ,axis=1)
        
        df.fillna(value='None',inplace=True)
        
    return df

# businessdf=pd.read_csv('business_table.csv')
l=['attributes']
newdf=create_attributes(newdf,l)

l=['BusinessParking', 'Ambience']
newdf=create_attributes(newdf,l)

In [15]:
#Dropping columns with lots of missing values
columns=['BestNights','DietaryRestrictions','Music','GoodForDancing','GoodForMeal',
       'CoatCheck', 'HappyHour', 'ByAppointmentOnly', 'BYOBCorkage',
       'DriveThru', 'WheelchairAccessible', 'BusinessAcceptsBitcoin',
       'Smoking', 'DogsAllowed', 'AcceptsInsurance', 'Corkage', 'BYOB',
       'AgesAllowed', 'HairSpecializesIn', 'Open24Hours',
       'RestaurantsCounterService','RestaurantsTableService', 'RestaurantsGoodForGroups','HasTV', 'BikeParking']
newdf.drop(columns, inplace=True, axis=1)


#Converting Parking column to True and False
newdf['Parking']=newdf['garage']
newdf['Parking']=np.where((newdf.garage==False) & (newdf.street==False) & (newdf.validated==False) & (newdf.lot==False)&(newdf.valet==False), False,newdf['Parking'])
newdf['Parking'] = np.where((newdf.garage==True) | (newdf.street==True) | (newdf.validated==True) | (newdf.lot==True)|(newdf.valet==True), True,newdf['Parking'])

columns=['garage', 'street','validated', 'lot', 'valet']
newdf.drop(columns, inplace=True, axis=1)

#Converting Alcohol column to True and False
newdf['Alcohol']=np.where((newdf.Alcohol==None) | (newdf.Alcohol=="u'none'") | (newdf.Alcohol=="'none'"), False,True)


#cleaning RestaurantsAttire Column
newdf['RestaurantsAttire']=newdf['RestaurantsAttire'].replace(["'casual'", "u'casual'"], 'casual')
newdf['RestaurantsAttire']=newdf['RestaurantsAttire'].replace(["'dressy'", "u'dressy'"], 'dressy')
newdf['RestaurantsAttire']=newdf['RestaurantsAttire'].replace(["'formal'", "u'formal'"], 'formal')

#Cleaning WiFi Column
newdf['WiFi']=np.where((newdf.WiFi==None) | (newdf.WiFi=="u'no'") | (newdf.WiFi=="'no'"), False,True)


#Cleaning NoiseLevel Column
newdf['Noise']=newdf['NoiseLevel']
newdf['Noise']=np.where((newdf.NoiseLevel=="u'quiet'") | (newdf.NoiseLevel=="'quiet'") ,'Low',newdf['Noise'])
newdf['Noise']=np.where((newdf.NoiseLevel=="u'average'") | (newdf.NoiseLevel=="'average'") ,'Average',newdf['Noise'])
newdf['Noise']=np.where((newdf.NoiseLevel=="u'very_loud'") | (newdf.NoiseLevel=="'very_loud'")| (newdf.NoiseLevel=="u'loud'")| (newdf.NoiseLevel=="'loud'") ,'High',newdf['Noise'])

columns=['NoiseLevel']
newdf.drop(columns, inplace=True, axis=1)

UNSTRUCTURED ANAYSIS

In [18]:
# Analysis of reviews on Restaurant with most number of reviews

biz_id_wc = '4JNXUYY8wbaaDmk3BPzlWw'
data_wc = reviewsdf.copy()
data_wc = data_wc.loc[data_wc['business_id']==biz_id_wc]
review_list = list(data_wc['review_text'])
review_list

["Hard to beat this location for table side entertainment. From people watching to the water show at the Belagio it's fun to watch while enjoying a great cocktail and dinner. See pics for our favorites!",
 'Sometimes, all a girl needs is a great steak.\n\nThis past year has been rather rough.\n\nDealing with professional and personal issues, including a relationship that involved the other cheating all over the virtual realm, so the escape to Sin City was an escape from all the stress of dealing with animate bs that usually accompanies the aforementioned.  I mean, what is one suppose to do after dealing with a person who was a chronic virtual cheater?  I attempt to forget.  \n\nI had a great time at Hakkasan the night before, so in continuing the positivity, I mentioned to my friend that a steak was in order.  After researching on Yelp mobile for a good yet affordable spot on the strip, Mon Ami Gabi popped up.  A smattering of the reviews were sarcastic regarding the location right on 

In [20]:

token_d, lemmatized_token_d, stop_words_removed= [],[],[]

# tokenize

for i in range(len(review_list)):
    d = review_list[i].lower()
    token_d.append(nltk.word_tokenize(d))

# lemmatize    

lemmatizer = nltk.stem.WordNetLemmatizer()
for token_list in token_d:
    temp=[]
    for token in token_list:
        if token.isalpha(): temp.append(lemmatizer.lemmatize(token))
    lemmatized_token_d.append(temp)
    
# removing stop words

for token_list in lemmatized_token_d:
    temp=[token for token in token_list if not token in stopwords.words('english') if token.isalpha()]
    stop_words_removed.append(temp)

# converting into tf-idf vectors. Min freq = half of the number of reviews

vectorizer2 = TfidfVectorizer(ngram_range=1, min_df=len(review_list)/2)

temp_list=[]
for token_list in stop_words_removed:
    temp_list.append(' '.join(token_list))

temp_list

['hard beat location table side entertainment people watching water show belagio fun watch enjoying great cocktail dinner see pic favorite',
 'sometimes girl need great steak past year ha rather rough dealing professional personal issue including relationship involved cheating virtual realm escape sin city wa escape stress dealing animate b usually accompanies aforementioned mean one suppose dealing person wa chronic virtual cheater attempt forget great time hakkasan night continuing positivity mentioned friend steak wa order researching yelp mobile good yet affordable spot strip mon ami gabi popped smattering review sarcastic regarding location right strip otherwise positive review photo accentuated friend previous positive experience mon ami gabi come first return visitor one consume food anywhere building great meal sight sound vega even better friend made reservation open via yelp mobile note section instructed friend state wanted seating outdoor patio closer railing imagine deligh

In [23]:
rating_list = list(data_wc['stars'])

word_wc, word_wc_rating = [],[]

for i in range(len(rating_list)):
    for j in range(len(stop_words_removed[i])):
        word_wc.append(stop_words_removed[i][j])
        word_wc_rating.append(rating_list[i])

d = {'word':word_wc,'word_rating':word_wc_rating}
df = pd.DataFrame(d)
n = df['word'].value_counts()
counts = {'word':list(n.index),'word_counts':list(n.values)}
counts = pd.DataFrame(counts)
df = df.groupby(['word'], as_index=False).agg({'word_rating':np.mean})
df = df.rename({'word_rating':'avg_word_rating'}, axis=1)

df = pd.merge(left=df, right=counts, left_on='word', right_on='word')
df['word_plus_rating'] = df['word'] + round(df['avg_word_rating'],1).astype(str)
df.sort_values(by='word_counts', ascending=False)

,word,avg_word_rating,word_counts,word_plus_rating
13844,wa,3.918424,25081,wa3.9
5062,food,3.984043,6079,food4.0
5647,good,4.001040,5772,good4.0
5758,great,4.322972,5598,great4.3
12157,steak,4.046894,4862,steak4.0
11252,service,4.067773,4279,service4.1
9399,place,4.143360,4262,place4.1
13650,vega,4.343188,3890,vega4.3
12965,time,4.119556,3605,time4.1
5215,french,4.124317,3475,french4.1


Analysis shows Food, Service and Time are key things people talk about. 
Next, we filter reviews data for Las Vegas and get reviews for latest two months as the dataset is huge

In [24]:
newdf1=businessdf[businessdf['city']=='Las Vegas']
business_id=newdf1['business_id'].tolist()

reviewsdf=reviewsdf.dropna()
reviewsdf=reviewsdf[reviewsdf['business_id'].isin(business_id)]
reviewsdf['review_date']= pd.to_datetime(reviewsdf['review_date'])
date_filter=reviewsdf[(reviewsdf['review_date'] > '2018-10-01')]

In [26]:
reviews=date_filter['review_text'].tolist()

f=["food"]
s=['service']
t=['time']

def checkword_food(sentence):
    words = nltk.word_tokenize(sentence)
    return any((True for word in words if word in f))

def checkword_service(sentence):
    words = nltk.word_tokenize(sentence)
    return any((True for word in words if word in s))

def checkword_time(sentence):
    words = nltk.word_tokenize(sentence)
    return any((True for word in words if word in t))


food=[]
service=[]
time=[]
for i in range(len(reviews)):
    print(i)
    if checkword_food(reviews[i]) is True:
        food.append('1')
    else:
        food.append('0')
        
    if checkword_service(reviews[i]) is True:
        service.append('1')
    else:
        service.append('0')
        
    if checkword_time(reviews[i]) is True:
        time.append('1')
    else:
        time.append('0')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262


3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3935
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3947
3948
3949
3950
3951
3952
3953
3954
3955
3956
3957
3958
3959
3960
3961
3962
3963
3964
3965
3966
3967
3968
3969
3970
3971
3972
3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082


5767
5768
5769
5770
5771
5772
5773
5774
5775
5776
5777
5778
5779
5780
5781
5782
5783
5784
5785
5786
5787
5788
5789
5790
5791
5792
5793
5794
5795
5796
5797
5798
5799
5800
5801
5802
5803
5804
5805
5806
5807
5808
5809
5810
5811
5812
5813
5814
5815
5816
5817
5818
5819
5820
5821
5822
5823
5824
5825
5826
5827
5828
5829
5830
5831
5832
5833
5834
5835
5836
5837
5838
5839
5840
5841
5842
5843
5844
5845
5846
5847
5848
5849
5850
5851
5852
5853
5854
5855
5856
5857
5858
5859
5860
5861
5862
5863
5864
5865
5866
5867
5868
5869
5870
5871
5872
5873
5874
5875
5876
5877
5878
5879
5880
5881
5882
5883
5884
5885
5886
5887
5888
5889
5890
5891
5892
5893
5894
5895
5896
5897
5898
5899
5900
5901
5902
5903
5904
5905
5906
5907
5908
5909
5910
5911
5912
5913
5914
5915
5916
5917
5918
5919
5920
5921
5922
5923
5924
5925
5926
5927
5928
5929
5930
5931
5932
5933
5934
5935
5936
5937
5938
5939
5940
5941
5942
5943
5944
5945
5946
5947
5948
5949
5950
5951
5952
5953
5954
5955
5956
5957
5958
5959
5960
5961
5962
5963
5964
5965
5966


7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507
7508
7509
7510
7511
7512
7513
7514
7515
7516
7517
7518
7519
7520
7521
7522
7523
7524
7525
7526
7527
7528
7529
7530
7531
7532
7533
7534
7535
7536
7537
7538
7539
7540
7541
7542
7543
7544
7545
7546
7547
7548
7549
7550
7551
7552
7553
7554
7555
7556
7557
7558
7559
7560
7561
7562
7563
7564
7565
7566
7567
7568
7569
7570
7571
7572
7573
7574
7575
7576
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7587
7588
7589
7590
7591
7592
7593
7594
7595
7596
7597
7598
7599
7600
7601
7602
7603
7604
7605
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
7618
7619
7620
7621
7622
7623
7624
7625
7626
7627
7628
7629
7630
7631
7632
7633
7634
7635
7636
7637
7638
7639
7640
7641
7642
7643
7644
7645
7646
7647
7648
7649
7650
7651
7652
7653
7654


...

22542
22543
22544
22545
22546
22547
22548
22549
22550
22551
22552
22553
22554
22555
22556
22557
22558
22559
22560
22561
22562
22563
22564
22565
22566
22567
22568
22569
22570
22571
22572
22573
22574
22575
22576
22577
22578
22579
22580
22581
22582
22583
22584
22585
22586
22587
22588
22589
22590
22591
22592
22593
22594
22595
22596
22597
22598
22599
22600
22601
22602
22603
22604
22605
22606
22607
22608
22609
22610
22611
22612
22613
22614
22615
22616
22617
22618
22619
22620
22621
22622
22623
22624
22625
22626
22627
22628
22629
22630
22631
22632
22633
22634
22635
22636
22637
22638
22639
22640
22641
22642
22643
22644
22645
22646
22647
22648
22649
22650
22651
22652
22653
22654
22655
22656
22657
22658
22659
22660
22661
22662
22663
22664
22665
22666
22667
22668
22669
22670
22671
22672
22673
22674
22675
22676
22677
22678
22679
22680
22681
22682
22683
22684
22685
22686
22687
22688
22689
22690
22691
22692
22693
22694
22695
22696
22697
22698
22699
22700
22701
22702
22703
22704
22705
22706
22707
2270

23917
23918
23919
23920
23921
23922
23923
23924
23925
23926
23927
23928
23929
23930
23931
23932
23933
23934
23935
23936
23937
23938
23939
23940
23941
23942
23943
23944
23945
23946
23947
23948
23949
23950
23951
23952
23953
23954
23955
23956
23957
23958
23959
23960
23961
23962
23963
23964
23965
23966
23967
23968
23969
23970
23971
23972
23973
23974
23975
23976
23977
23978
23979
23980
23981
23982
23983
23984
23985
23986
23987
23988
23989
23990
23991
23992
23993
23994
23995
23996
23997
23998
23999
24000
24001
24002
24003
24004
24005
24006
24007
24008
24009
24010
24011
24012
24013
24014
24015
24016
24017
24018
24019
24020
24021
24022
24023
24024
24025
24026
24027
24028
24029
24030
24031
24032
24033
24034
24035
24036
24037
24038
24039
24040
24041
24042
24043
24044
24045
24046
24047
24048
24049
24050
24051
24052
24053
24054
24055
24056
24057
24058
24059
24060
24061
24062
24063
24064
24065
24066
24067
24068
24069
24070
24071
24072
24073
24074
24075
24076
24077
24078
24079
24080
24081
24082
2408

25333
25334
25335
25336
25337
25338
25339
25340
25341
25342
25343
25344
25345
25346
25347
25348
25349
25350
25351
25352
25353
25354
25355
25356
25357
25358
25359
25360
25361
25362
25363
25364
25365
25366
25367
25368
25369
25370
25371
25372
25373
25374
25375
25376
25377
25378
25379
25380
25381
25382
25383
25384
25385
25386
25387
25388
25389
25390
25391
25392
25393
25394
25395
25396
25397
25398
25399
25400
25401
25402
25403
25404
25405
25406
25407
25408
25409
25410
25411
25412
25413
25414
25415
25416
25417
25418
25419
25420
25421
25422
25423
25424
25425
25426
25427
25428
25429
25430
25431
25432
25433
25434
25435
25436
25437
25438
25439
25440
25441
25442
25443
25444
25445
25446
25447
25448
25449
25450
25451
25452
25453
25454
25455
25456
25457
25458
25459
25460
25461
25462
25463
25464
25465
25466
25467
25468
25469
25470
25471
25472
25473
25474
25475
25476
25477
25478
25479
25480
25481
25482
25483
25484
25485
25486
25487
25488
25489
25490
25491
25492
25493
25494
25495
25496
25497
25498
2549

26897
26898
26899
26900
26901
26902
26903
26904
26905
26906
26907
26908
26909
26910
26911
26912
26913
26914
26915
26916
26917
26918
26919
26920
26921
26922
26923
26924
26925
26926
26927
26928
26929
26930
26931
26932
26933
26934
26935
26936
26937
26938
26939
26940
26941
26942
26943
26944
26945
26946
26947
26948
26949
26950
26951
26952
26953
26954
26955
26956
26957
26958
26959
26960
26961
26962
26963
26964
26965
26966
26967
26968
26969
26970
26971
26972
26973
26974
26975
26976
26977
26978
26979
26980
26981
26982
26983
26984
26985
26986
26987
26988
26989
26990
26991
26992
26993
26994
26995
26996
26997
26998
26999
27000
27001
27002
27003
27004
27005
27006
27007
27008
27009
27010
27011
27012
27013
27014
27015
27016
27017
27018
27019
27020
27021
27022
27023
27024
27025
27026
27027
27028
27029
27030
27031
27032
27033
27034
27035
27036
27037
27038
27039
27040
27041
27042
27043
27044
27045
27046
27047
27048
27049
27050
27051
27052
27053
27054
27055
27056
27057
27058
27059
27060
27061
27062
2706

28280
28281
28282
28283
28284
28285
28286
28287
28288
28289
28290
28291
28292
28293
28294
28295
28296
28297
28298
28299
28300
28301
28302
28303
28304
28305
28306
28307
28308
28309
28310
28311
28312
28313
28314
28315
28316
28317
28318
28319
28320
28321
28322
28323
28324
28325
28326
28327
28328
28329
28330
28331
28332
28333
28334
28335
28336
28337
28338
28339
28340
28341
28342
28343
28344
28345
28346
28347
28348
28349
28350
28351
28352
28353
28354
28355
28356
28357
28358
28359
28360
28361
28362
28363
28364
28365
28366
28367
28368
28369
28370
28371
28372
28373
28374
28375
28376
28377
28378
28379
28380
28381
28382
28383
28384
28385
28386
28387
28388
28389
28390
28391
28392
28393
28394
28395
28396
28397
28398
28399
28400
28401
28402
28403
28404
28405
28406
28407
28408
28409
28410
28411
28412
28413
28414
28415
28416
28417
28418
28419
28420
28421
28422
28423
28424
28425
28426
28427
28428
28429
28430
28431
28432
28433
28434
28435
28436
28437
28438
28439
28440
28441
28442
28443
28444
28445
2844

In [27]:
date_filter.columns

Index(['Unnamed: 0', 'review_id', 'user_id', 'business_id', 'stars',
       'review_text', 'review_date'],
      dtype='object')

In [28]:
date_filter['Food'] = food
date_filter['Service'] = service
date_filter['Time'] = time


# date_filter.columns
columns=['Unnamed: 0',  'review_id', 'user_id',
        'review_text', 'review_date']
date_filter.drop(columns, inplace=True, axis=1)


date_filter.Food = date_filter.Food. astype(int)
date_filter.Service = date_filter.Service. astype(int)
date_filter.Time = date_filter.Time. astype(int)

agg=date_filter.groupby(['business_id'], as_index=False).mean()

C:\Users\prana\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\prana\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\prana\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

MERGING THE TWO DATASETS TO GET FINAL DATASET

In [36]:
dataset = pd.merge(left=newdf,right=agg,left_on='business_id',right_on='business_id')

In [37]:
dataset

,Unnamed: 0,business_id,business_name,business_address,city,state,postal_code,stars_x,review_count,GoodForKids,...,classy,trendy,upscale,casual,Parking,Noise,stars_y,Food,Service,Time
0,25,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,Las Vegas,NV,89156,4.5,184,True,...,False,False,False,True,True,Average,5.000000,0.750000,0.500000,0.000000
1,128,sKhDrZFCJqfRNylkHrIDsQ,Starbucks,"1990 Village Center Circle, Suite 1",Las Vegas,NV,89134,4.0,76,None,...,None,None,None,None,False,None,5.000000,0.000000,1.000000,1.000000
2,174,6fPQJq4f_yiq1NHn0fd11Q,La Creperie,3655 Las Vegas Blvd S,Las Vegas,NV,89109,3.5,535,True,...,False,False,False,True,True,Average,4.000000,0.166667,0.333333,0.166667
3,176,k-dDZvTeLysoJvjHI-qr9g,Feast Buffet,2411 W Sahara Ave,Las Vegas,NV,89102,3.0,287,True,...,False,False,False,True,True,Average,3.038462,0.692308,0.269231,0.192308
4,494,k2b3niokS_tosjah_rzCPw,Metro Pizza,6720 Sky Pointe Dr,Las Vegas,NV,89131,3.5,303,True,...,False,False,False,True,True,Average,3.636364,0.272727,0.181818,0.000000
5,693,hgQOESYp7CF4LnNI90XEiw,800 Degrees,2535 Las Vegas Blvd S,Las Vegas,NV,89109,3.5,296,True,...,False,False,False,True,True,Average,4.500000,0.000000,0.000000,0.500000
6,718,F06m2yQSPHIrb1IT7heYeQ,Rainbow Kitchen,7537 S Rainbow Blvd,Las Vegas,NV,89139,4.0,101,True,...,False,False,False,True,True,Average,4.000000,0.666667,0.000000,0.333333
7,801,H2Chxto2e6dHTDJ8-s3-pQ,Roberto's Taco Shop,2685 S Eastern Ave,Las Vegas,NV,89169,3.0,64,True,...,False,False,False,True,True,Average,2.000000,1.000000,0.000000,1.000000
8,929,yKpdzrqe_qIbu9dB6bj9Gg,PGA Tour Grill,5757 Wayne Newton Blvd,Las Vegas,NV,89119,2.5,80,False,...,False,False,False,True,False,None,2.333333,0.333333,0.333333,0.166667
9,941,lXCmTGagqHuPgaeN2g2P1Q,Marcos Pizza,5061 E Sahara Ave,Las Vegas,NV,89142,3.5,39,True,...,False,False,False,True,False,Average,2.000000,0.000000,0.000000,0.000000


LINEAR REGRESSION

In [38]:
#create dummies
dataset = pd.get_dummies(dataset, columns=['GoodForKids',
       'RestaurantsDelivery', 'Alcohol', 'Caters', 'WiFi',
       'RestaurantsTakeOut', 'BusinessAcceptsCreditCards', 'OutdoorSeating',
       'RestaurantsReservations', 'RestaurantsPriceRange2',
       'RestaurantsAttire', 'romantic', 'intimate', 'touristy', 'hipster',
       'divey', 'classy', 'trendy', 'upscale', 'casual', 'Parking', 'Noise'], drop_first=True)

In [39]:
target=['stars_x']
variables=['review_count',
       'GoodForKids_None', 'GoodForKids_True', 'RestaurantsDelivery_None',
       'RestaurantsDelivery_True', 'Alcohol_True', 'Caters_None',
       'Caters_True', 'WiFi_True', 'RestaurantsTakeOut_None',
       'RestaurantsTakeOut_True', 'BusinessAcceptsCreditCards_None',
       'BusinessAcceptsCreditCards_True', 'OutdoorSeating_None',
       'OutdoorSeating_True', 'RestaurantsReservations_None',
       'RestaurantsReservations_True', 'RestaurantsPriceRange2_2',
       'RestaurantsPriceRange2_3', 'RestaurantsPriceRange2_4',
       'RestaurantsPriceRange2_None', 'RestaurantsAttire_casual',
       'RestaurantsAttire_dressy', 'RestaurantsAttire_formal', 'romantic_True',
       'romantic_None', 'intimate_True', 'intimate_None', 'touristy_True',
       'touristy_None', 'hipster_True', 'hipster_None', 'divey_True',
       'divey_None', 'classy_True', 'classy_None', 'trendy_True',
       'trendy_None', 'upscale_True', 'upscale_None', 'casual_True',
       'casual_None', 'Parking_True', 'Parking_None', 'Noise_High',
       'Noise_Low', 'Noise_None','Food','Service','Time']


X = dataset[variables]
y = dataset[target]

In [43]:
model = sm.OLS(y, X).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                stars_x   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                     2118.
Date:                Sun, 01 Dec 2019   Prob (F-statistic):               0.00
Time:                        14:50:45   Log-Likelihood:                -4555.7
No. Observations:                3991   AIC:                             9195.
Df Residuals:                    3949   BIC:                             9460.
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
review_count                        0.0002   2.74e-05      6.664      0.000       0.000       0.000
GoodForKids_None                    0.4046      0.067      6.083      0.000       0.274       0.535
GoodForKids_True                    0.1476      0.044      3.335      0.001       0.061       0.234
RestaurantsDelivery_None            0.1373      0.059      2.313      0.021       0.021       0.254
RestaurantsDelivery_True            0.0889      0.036      2.485      0.013       0.019       0.159
Alcohol_True                        0.1523      0.034      4.450      0.000       0.085       0.219
Caters_None                         0.1859      0.048      3.839      0.000       0.091       0.281
Caters_True                         0.2491      0.031      7.965      0.000       0.188       0.310
WiFi_True                           0.0735      0.028      2.651      0.008       0.019       0.128
RestaurantsTakeOut_None             0.5195      0.076      6.833      0.000       0.370       0.669
RestaurantsTakeOut_True             0.3890      0.047      8.357      0.000       0.298       0.480
BusinessAcceptsCreditCards_None     1.5566      0.133     11.670      0.000       1.295       1.818
BusinessAcceptsCreditCards_True     1.6475      0.081     20.406      0.000       1.489       1.806
OutdoorSeating_None                 0.1206      0.056      2.156      0.031       0.011       0.230
OutdoorSeating_True                 0.0955      0.030      3.183      0.001       0.037       0.154
RestaurantsReservations_None        0.1472      0.081      1.816      0.069      -0.012       0.306
RestaurantsReservations_True        0.1949      0.036      5.483      0.000       0.125       0.265
RestaurantsPriceRange2_2            0.0263      0.031      0.843      0.399      -0.035       0.087
RestaurantsPriceRange2_3            0.3464      0.077      4.516      0.000       0.196       0.497
RestaurantsPriceRange2_4            0.7934      0.141      5.617      0.000       0.516       1.070
RestaurantsPriceRange2_None         0.2991      0.074      4.022      0.000       0.153       0.445
RestaurantsAttire_casual            0.3214      0.069      4.624      0.000       0.185       0.458
RestaurantsAttire_dressy            0.3958      0.125      3.173      0.002       0.151       0.640
RestaurantsAttire_formal            0.8044      0.766      1.050      0.294      -0.698       2.307
romantic_True                       0.0943      0.109      0.866      0.386      -0.119       0.308
romantic_None                      -0.0029      0.009     -0.333      0.739      -0.020       0.014
intimate_True                       0.2993      0.098      3.065      0.002       0.108       0.491
intimate_None                      -0.0029      0.009     -0.333      0.739      -0